<img src="https://www3.um.edu.uy/logoum.jpg" width=300>
    <h1 align="center">Thesis - Alternating Least Squares hyperparameter optimization</h1> 
<h2 align="center">Alejo Paullier</h2> 

# Import libraries

In [7]:
import pandas as pd
import numpy as np
import scipy
import sklearn
import flask
import GPyOpt
import sherpa
import pylab as pl
import time

from lenskit import batch, topn, util
from lenskit import crossfold as xf
from lenskit.algorithms import Recommender, als
from lenskit import topn

# Import data & define evaluation function

In [2]:
ratings = pd.read_csv('C:\\Users\\Alejo\\Tesis\\Demo\\ml-100k\\u.data', sep='\t',
                      names=['user', 'item', 'rating', 'timestamp'])


def eval(aname, algo, train, test):
    fittable = util.clone(algo) # Object cloning means to create an exact copy of the original object.
    fittable = Recommender.adapt(fittable) 
    fittable.fit(train) # Entrenamos el algoritmo con el training dataset
    users = test.user.unique() # Devuelve un array con los users unicos en el testing dataset
    # now we run the recommender
    recs = batch.recommend(fittable, users, 100)
    # add the algorithm name for analyzability
    recs['Algorithm'] = aname
    return recs

# Define model

In [3]:
def models(feat):
    all_recs = []
    test_data = []
    algo_als = als.BiasedMF(feat,iterations=20, reg=0.1, damping=5, bias=True, progress=None)
    for train, test in xf.partition_users(ratings[['user', 'item', 'rating']], 1, xf.SampleFrac(0.2)):
        test_data.append(test)
        all_recs.append(eval('ALS', algo_als, train, test))
    all_recs = pd.concat(all_recs, ignore_index=True)
    all_recs.head()
    test_data = pd.concat(test_data, ignore_index=True)
    rla = topn.RecListAnalysis()
    rla.add_metric(topn.ndcg)
    results = rla.compute(all_recs, test_data)
    results.head()
    ndcg = results.groupby('Algorithm').ndcg.mean() 
    return ndcg

# Define parameters

In [8]:
parameters = [sherpa.Discrete('feat', [1,20])]
alg = sherpa.algorithms.RandomSearch(max_num_trials=10)

In [9]:
study = sherpa.Study(parameters=parameters,
                     algorithm=alg,
                     lower_is_better=False,
                     disable_dashboard=True)

In [10]:
for trial in study:
    print("Trial {}:\t{}".format(trial.id, trial.parameters))
    model_trainer = models(trial.parameters['feat'])
    num_iterations = 10
    for iteration in range(num_iterations):
        study.add_observation(trial=trial,
                              iteration=iteration,
                              objective=model_trainer)
        time.sleep(0.1)
    study.finalize(trial=trial)

Trial 1:	{'feat': 19}


NameError: name 'models' is not defined

In [ ]:
help("modules")

#### environment variable
An environment variable is a key, value pair that is stored in computer memory and available to other programs running in the same environment. For example the PATH environment variable, is a key, value pair where the key is PATH and the value is a list of directories, such as /usr/bin:/bin:/usr/sbin:/sbin:/usr/local/bin. In particular, the shell uses the value of the PATH environment variable as a list of directories to search for executable programs.